# TPS Oct 2021

A "Getting Started" LGBM notebook.

Save train_pred_2.csv and test_pred_2.csv for blending.

## Others

- [TPS Oct 2021 - Basic XGBoost CV](https://www.kaggle.com/mmellinger66/tps-oct-2021-basic-xgboost-cv) - train_pred_1.csv / test_pred_1.csv for blending.

## Versions

- V9: Try StratifiedKFold
- V8: Started added Blending code
- V7: Different parameters
- V6: KFold
- V5: ...

# References

- [LGBM-OneModel-ThreeSeeds-Blend](https://www.kaggle.com/mlanhenke/tps-10-lgbm-onemodel-threeseeds-blend)
- [LightGBM Classifier in Python](https://www.kaggle.com/prashant111/lightgbm-classifier-in-python)
- [Simple LightGBM Model](https://www.kaggle.com/gorosia/simple-lightgbm-model)
- [competition part-5: blending 101](https://www.kaggle.com/abhishek/competition-part-5-blending-101)


# Load Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

from pathlib import Path


# Configuration

In [ ]:
class Config:
    debug = False
    competition = "PetFinder"
    seed = 42
    NFOLDS = 5
    EPOCHS = 10
    

In [ ]:
data_dir = Path('../input/tabular-playground-series-oct-2021') # Change me every month

# Load Data

In [ ]:
train_df = pd.read_csv(data_dir / "train.csv",
#                       nrows=100000
                      )

test_df = pd.read_csv(data_dir / "test.csv")
sample_submission = pd.read_csv(data_dir / "sample_submission.csv")

print(f"train data: Rows={train_df.shape[0]}, Columns={train_df.shape[1]}")
print(f"test data : Rows={test_df.shape[0]}, Columns={test_df.shape[1]}")

In [ ]:
train_df.info()

In [ ]:
train_df.head()

# Reduce Memory

Too many memory issues.  Got a function to reduce the float and int types by checking the max column value and setting column to minimum necessary type.

- https://www.kaggle.com/hrshuvo/tps-oct-21-xgb-kfold
- https://www.kaggle.com/rinnqd/reduce-memory-usage

In [ ]:
# this function will help to reduce momory 
# data will be smaller with the same value

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

# Extract Target

In [ ]:
y = train_df.target

test_df = test_df.drop(["id"], axis=1)
X = train_df.drop(["id", "target"], axis=1)
X.head()

In [ ]:
# Train/Validation Split
# x_train, x_valid, y_train, y_valid = train_test_split(X, y,
#                                                     test_size=0.2,
#                                                     stratify=None,
#                                                     random_state=42)


# Feature Engineering

In [ ]:
features = [col for col in train_df.columns if col not in ('id', 'target')]

# Model

In [ ]:
# TODO: Find good defaults

lgbm_params = {
    'objective': 'binary',
    'num_leaves': 20,
    'max_depth': 7,
    'n_estimators': 200,
    'metric': 'AUC',
    'learning_rate': 0.05
}

In [ ]:
lgbm_params = {
    'objective': 'binary', 
    'device_type': 'gpu', 
    'n_estimators': 20000, 
    'learning_rate':  0.01, 
    'min_child_weight': 256,
    'min_child_samples': 20, 
    'reg_alpha': 10, 
    'reg_lambda': 0.1, 
    'subsample': 0.6, 
    'subsample_freq': 1, 
    'colsample_bytree': 0.4,
}

In [ ]:
final_test_predictions = []
final_valid_predictions = {}
scores = []

kf = StratifiedKFold(n_splits=Config.NFOLDS, shuffle=True, random_state=Config.seed)

for fold, (train_idx, valid_idx) in enumerate(kf.split(X = X, y = y)):
#     print(type(train_idx))
    print(10*"=", f"Fold={fold}", 10*"=")

    x_train = X.loc[train_idx, :]
    x_valid = X.loc[valid_idx, :]
    
    y_train = y[train_idx]
    y_valid = y[valid_idx]
    model = lgb.LGBMClassifier(**lgbm_params)
    model.fit(x_train, y_train,
                eval_set=[(x_valid, y_valid)],
              eval_metric='auc',
              early_stopping_rounds=100,
              verbose=0)
    
    preds_valid = model.predict_proba(x_valid)[:, -1]
    # Want probability or classification?
    final_valid_predictions.update(dict(zip(valid_idx, preds_valid)))

    auc = roc_auc_score(y_valid, preds_valid)
    print('auc: ', auc)
    scores.append(auc)
    
    test_preds = model.predict_proba(test_df[features])[:, -1]
    final_test_predictions.append(test_preds)


In [ ]:
print(f"scores -> mean: {np.mean(scores)}, std: {np.std(scores)}")

# Save OOF Predictions

Save the dictionary that we created for all the training predictions that were made when each fold was used for validation

In [ ]:
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("train_pred_2.csv", index=False)

# Predict

In [ ]:
# predictions = model.predict_proba(test_df)

In [ ]:
# predictions = predictions[:,1] # Forgot this!

In [ ]:
df = pd.DataFrame(np.column_stack(final_test_predictions))
df['mean'] = df.mean(axis=1)
df

# Submission

In [ ]:
sample_submission['target'] = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.to_csv("test_pred_2.csv",index=None)
sample_submission.to_csv("basic_lgbm_cv.csv",index=None)
sample_submission